In [ ]:
# == Preparação: copie este bloco no Google Colab e execute ==
# Se quiser salvar no Google Drive, descomente a seção de mount.

!pip install faker --quiet

import os
import numpy as np
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta

# ---------- CONFIGURAÇÃO ----------
RANDOM_SEED = 42
N_FACT_ROWS = 500_000   # número de linhas da tabela fato
N_CLIENTS = 5_000       # número de clientes (ajuste se quiser)
N_PRODUCTS = 1_000      # número de produtos
N_REPS = 200            # número de representantes
MONTHS_SPAN = 24        # janela temporal (meses)

OUTPUT_DIR = "/content/powerbi_sample"  # pasta de saída no Colab
os.makedirs(OUTPUT_DIR, exist_ok=True)

np.random.seed(RANDOM_SEED)
fake = Faker('pt_BR')
Faker.seed(RANDOM_SEED)

# ---------- Dimensão: Produtos ----------
# Cria produtos com preços base (skewed)
product_ids = [f"P{str(i).zfill(4)}" for i in range(1, N_PRODUCTS + 1)]
product_names = [f"Produto {i}" for i in range(1, N_PRODUCTS + 1)]

# Categorias - exemplo
categorias = ["Eletrônicos", "Móveis", "Alimentos", "Vestuário", "Beleza", "Ferramentas"]
prod_categoria = np.random.choice(categorias, size=N_PRODUCTS, p=[0.18,0.15,0.20,0.18,0.14,0.15])

# Preço base por produto (log-normal para simular variação)
price_base = np.round(np.random.lognormal(mean=3.5, sigma=0.8, size=N_PRODUCTS), 2)  # média ~ exp(3.5)
preco_unitario = price_base

dim_produtos = pd.DataFrame({
    "ID_Produto": product_ids,
    "Nome_Produto": product_names,
    "Categoria": prod_categoria,
    "Preco_Base": preco_unitario
})

# ---------- Dimensão: Clientes ----------
client_ids = [f"C{str(i).zfill(5)}" for i in range(1, N_CLIENTS + 1)]
client_names = [fake.company() for _ in range(N_CLIENTS)]

# Segmento e Região
segmentos = ["Varejo", "Atacado", "Distribuidor", "E-commerce"]
regioes = ["Norte", "Nordeste", "Centro-Oeste", "Sudeste", "Sul"]

# Probabilidades para tipos de cliente (pode ser alterado)
seg_probs = [0.45, 0.25, 0.15, 0.15]
reg_probs = [0.08, 0.15, 0.07, 0.50, 0.20]

client_segmento = np.random.choice(segmentos, size=N_CLIENTS, p=seg_probs)
client_regiao = np.random.choice(regioes, size=N_CLIENTS, p=reg_probs)

# Score de atividade (para simular clientes mais/menos ativos)
activity_score = np.random.beta(a=2, b=5, size=N_CLIENTS)  # 0..1

dim_clientes = pd.DataFrame({
    "ID_Cliente": client_ids,
    "Nome_Cliente": client_names,
    "Segmento": client_segmento,
    "Regiao": client_regiao,
    "Activity_Score": np.round(activity_score, 3)
})

# ---------- Dimensão: Representantes ----------
rep_ids = [f"R{str(i).zfill(3)}" for i in range(1, N_REPS + 1)]
rep_names = [fake.name() for _ in range(N_REPS)]
rep_regiao = np.random.choice(regioes, size=N_REPS, p=reg_probs)

dim_reps = pd.DataFrame({
    "ID_Representante": rep_ids,
    "Nome_Representante": rep_names,
    "Regiao": rep_regiao
})

# ---------- Geração da FATO: Vendas ----------
# Datas: geraremos datas uniformes nos últimos MONTHS_SPAN meses, mas ponderadas por Activity_Score do cliente
today = datetime.today().date()
start_date = (today - pd.DateOffset(months=MONTHS_SPAN)).date()

# 1) escolher clientes para cada linha da fato, ponderando por Activity_Score
client_probs = dim_clientes["Activity_Score"].values
client_probs = client_probs / client_probs.sum()  # normaliza

fact_client_idx = np.random.choice(N_CLIENTS, size=N_FACT_ROWS, p=client_probs)
fact_client_ids = dim_clientes["ID_Cliente"].values[fact_client_idx]

# 2) escolher produto (pode ter pesos por categoria se quiser)
product_probs = np.ones(N_PRODUCTS) / N_PRODUCTS
fact_product_idx = np.random.choice(N_PRODUCTS, size=N_FACT_ROWS, p=product_probs)
fact_product_ids = dim_produtos["ID_Produto"].values[fact_product_idx]
fact_product_prices = dim_produtos["Preco_Base"].values[fact_product_idx]

# 3) representante (ops: representante prefere atuar na mesma região do cliente? vamos misturar)
# Vamos associar representantes preferencialmente à região do cliente (70%) ou aleatório (30%)
# Primeiro mapeia cliente -> região, escolhe reps da mesma região quando possível.
client_regions = dim_clientes["Regiao"].values
rep_regions = dim_reps["Regiao"].values

fact_rep_ids = []
for cidx in fact_client_idx:
    creg = client_regions[cidx]
    # reps in same region
    same_region_reps = dim_reps[dim_reps["Regiao"] == creg]["ID_Representante"].values
    if len(same_region_reps) > 0 and np.random.rand() < 0.7:
        fact_rep_ids.append(np.random.choice(same_region_reps))
    else:
        fact_rep_ids.append(np.random.choice(dim_reps["ID_Representante"].values))
fact_rep_ids = np.array(fact_rep_ids)

# 4) Quantidade: distribuição skewed (muitos itens pequenos, poucos grandes)
# Usamos Poisson + cap
quantidade = np.random.poisson(lam=3, size=N_FACT_ROWS) + 1
quantidade = np.clip(quantidade, 1, 200)

# 5) Preço unitário: Preço base do produto com volatilidade (promoções/variações)
# adicionar variação percentual -10%..+20%, com maior probabilidade perto de 0
pct_variation = np.random.normal(loc=0.02, scale=0.08, size=N_FACT_ROWS)  # média +2%
preco_unitario_fact = np.round(fact_product_prices * (1 + pct_variation), 2)
preco_unitario_fact = np.where(preco_unitario_fact <= 0.01, fact_product_prices, preco_unitario_fact)

# 6) Desconto (opcional): desconto em % quando quantidade alta
desconto_pct = np.where(quantidade >= 20, np.random.uniform(0.05, 0.20, size=N_FACT_ROWS),
                        np.random.uniform(0.00, 0.05, size=N_FACT_ROWS))
# aplicar desconto ao total
valor_bruto = quantidade * preco_unitario_fact
valor_total = np.round(valor_bruto * (1 - desconto_pct), 2)

# 7) Datas de venda: gerar datas com distribuição ao longo do período
# Vamos gerar dias aleatórios entre start_date e today com densidade uniforme
date_range_days = (today - start_date).days
random_days = np.random.randint(0, date_range_days + 1, size=N_FACT_ROWS)
datas_venda = pd.to_datetime([start_date + timedelta(days=int(d)) for d in random_days]).date

# 8) Criar IDs de venda (opcional)
venda_ids = [f"V{str(i).zfill(7)}" for i in range(1, N_FACT_ROWS + 1)]

# 9) Construir DataFrame fato
fato_vendas = pd.DataFrame({
    "ID_Venda": venda_ids,
    "Data_Venda": pd.to_datetime(datas_venda),
    "ID_Cliente": fact_client_ids,
    "ID_Produto": fact_product_ids,
    "ID_Representante": fact_rep_ids,
    "Quantidade": quantidade,
    "Preco_Unitario": preco_unitario_fact,
    "Desconto_Pct": np.round(desconto_pct, 4),
    "Valor_Total": valor_total
})

# ---------- Quick checks ----------
print("Dim Produtos:", dim_produtos.shape)
print("Dim Clientes:", dim_clientes.shape)
print("Dim Reps:", dim_reps.shape)
print("Fato Vendas:", fato_vendas.shape)
print("Exemplo linhas fato:\n", fato_vendas.head(5))

# ---------- Salvando CSVs ----------
dim_produtos.to_csv(os.path.join(OUTPUT_DIR, "dim_produtos.csv"), index=False)
dim_clientes.to_csv(os.path.join(OUTPUT_DIR, "dim_clientes.csv"), index=False)
dim_reps.to_csv(os.path.join(OUTPUT_DIR, "dim_representantes.csv"), index=False)
fato_vendas.to_csv(os.path.join(OUTPUT_DIR, "fato_vendas_500k.csv"), index=False)

print("\nArquivos salvos em:", OUTPUT_DIR)
print("dim_produtos.csv, dim_clientes.csv, dim_representantes.csv, fato_vendas_500k.csv")

Dim Produtos: (1000, 4)
Dim Clientes: (5000, 5)
Dim Reps: (200, 3)
Fato Vendas: (500000, 9)
Exemplo linhas fato:
    ID_Venda Data_Venda ID_Cliente ID_Produto ID_Representante  Quantidade  \
0  V0000001 2024-09-19     C01047      P0412             R189           3   
1  V0000002 2023-12-05     C04162      P0980             R176           5   
2  V0000003 2024-08-23     C03119      P0421             R029           5   
3  V0000004 2024-02-03     C03279      P0163             R099           4   
4  V0000005 2023-11-27     C02530      P0688             R160           3   

   Preco_Unitario  Desconto_Pct  Valor_Total  
0            9.18        0.0108        27.24  
1          107.73        0.0009       538.19  
2           35.59        0.0160       175.11  
3           60.07        0.0369       231.41  
4           54.09        0.0197       159.07  

Arquivos salvos em: /content/powerbi_sample
dim_produtos.csv, dim_clientes.csv, dim_representantes.csv, fato_vendas_500k.csv
